## Importar librerías

In [55]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster
import warnings
warnings.filterwarnings('ignore')

In [56]:
# Cargar
tester_files_folder = r"C:\Users\Administrador\AppData\Roaming\MetaQuotes\Terminal\Common\Files"
df_buy = pd.read_csv(os.path.join(tester_files_folder, "buy_dataset.csv"))
df_sell = pd.read_csv(os.path.join(tester_files_folder, "sell_dataset.csv"))
print(f"Buy -> Trades: {df_buy.shape[0]} Features: {df_buy.shape[1]-1}")
print(f"Sell -> Trades: {df_sell.shape[0]} Features: {df_sell.shape[1]-1}")

Buy -> Trades: 5202 Features: 58
Sell -> Trades: 4226 Features: 58


In [57]:
df_buy

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_49,data_50,data_51,data_52,data_53,data_54,data_55,data_56,data_57,target
0,0.598339,-0.367041,43.793228,0.008131,0.001569,-37400.0,-0.004653,0.007874,-0.166234,0.005502,...,-0.296569,501.685073,-0.034721,0.030500,1407009.0,-0.005359,-0.006926,0.416667,0.333333,1.0
1,-0.202430,0.329011,12.475479,0.003116,0.000510,-10433.0,-0.001281,0.002546,-0.344229,-0.247929,...,0.857648,159.519790,-0.008178,0.017017,1540848.0,0.037470,-0.011381,0.916667,0.500000,1.0
2,-0.635128,0.238026,2.054210,0.000267,0.000095,31649.0,-0.000125,0.000187,-0.094160,-0.162109,...,0.581907,197.942022,0.016502,0.012081,1259139.0,-0.036069,0.027665,0.291667,0.666667,1.0
3,-0.486586,0.016132,1.739963,0.000125,0.000094,-21003.0,-0.000230,0.000201,-0.555113,0.991905,...,-0.251307,150.184362,-0.025160,0.008398,2176925.0,0.014279,-0.019923,0.333333,0.833333,0.0
4,0.280515,0.565093,9.382670,0.001575,0.000495,17112.0,-0.001451,0.001158,-0.111756,0.045146,...,1.598165,368.496172,-0.081870,0.019557,1086559.0,0.072600,-0.056711,0.458333,0.833333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5197,0.085769,0.929304,2.507133,0.000527,0.000151,34683.0,-0.000186,0.000445,-0.958018,2.081794,...,1.669982,69.725033,-0.003850,0.008513,1786552.0,0.011547,0.001646,0.833333,0.666667,1.0
5198,-0.215181,-0.975171,7.871722,0.001620,0.000622,-5377.0,-0.001596,0.001531,0.686668,1.226102,...,0.852087,301.900586,0.014616,0.030063,1018792.0,0.079735,0.018875,0.500000,0.166667,0.0
5199,-0.558662,-0.383466,2.091322,0.000236,0.000059,33612.0,-0.000049,0.000224,0.859834,0.568266,...,2.758663,54.144434,-0.005549,0.008251,1750577.0,0.013954,-0.006451,0.250000,0.166667,0.0
5200,-0.084055,0.311404,2.736887,0.000715,0.000272,47674.0,-0.000224,0.000563,-0.199505,1.074852,...,3.206830,103.269215,0.017445,0.010020,1084811.0,0.008553,0.008203,0.375000,0.333333,1.0


In [58]:
df_sell

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_49,data_50,data_51,data_52,data_53,data_54,data_55,data_56,data_57,target
0,0.078904,-0.673371,14.836103,-0.002024,0.000772,-42577.0,0.001298,-0.001684,1.426781,2.617865,...,-0.298077,194.920112,0.021717,0.018932,1440129.0,-0.023815,0.009826,0.375000,0.333333,1.0
1,-0.199800,0.884652,8.168477,-0.001258,0.000644,-56019.0,0.000255,-0.000841,-0.485330,0.956994,...,-0.346676,534.031898,0.080191,0.032431,1095169.0,-0.087460,0.062388,0.625000,0.666667,1.0
2,0.509142,-0.307327,7.270488,-0.001291,0.000543,60610.0,0.000748,-0.001067,1.103104,1.642185,...,0.069439,126.947333,-0.009532,0.013023,1699854.0,0.026062,-0.001405,0.416667,0.333333,0.0
3,0.194632,-0.353290,12.864740,-0.003129,0.000782,-3605.0,0.002145,-0.002494,-0.771614,0.981393,...,0.466628,205.559475,-0.038619,0.028281,823818.0,0.045760,-0.032716,0.708333,0.333333,1.0
4,0.177534,-0.818139,1.195803,-0.000152,0.000065,57561.0,0.000166,-0.000146,-0.433564,0.031238,...,0.193424,145.170125,-0.019230,0.009070,1786854.0,0.020246,-0.013808,0.291667,0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4221,0.471591,-0.834042,5.806874,-0.000891,0.000286,47399.0,0.001195,-0.000790,-0.833083,0.585863,...,0.317880,116.675122,-0.003347,0.010999,1545417.0,-0.009497,-0.007635,0.666667,0.666667,0.0
4222,0.442555,-0.583857,36.793114,-0.007841,0.001442,-2076.0,0.004832,-0.006546,1.567046,2.614808,...,-0.643876,789.777538,0.152328,0.028155,421491.0,-0.105427,0.128986,0.416667,0.833333,1.0
4223,0.750410,-0.413239,3.606504,-0.000448,0.000083,41316.0,0.000309,-0.000406,0.893736,0.421593,...,0.540489,222.526139,-0.034877,0.009238,1673496.0,0.065823,-0.038046,0.291667,0.166667,0.0
4224,1.397731,2.229597,25.191901,-0.009229,0.001414,-17825.0,0.001958,-0.005337,0.642473,-0.008252,...,-0.784646,1403.584447,0.332995,0.063564,279406.0,-0.415117,0.249132,0.416667,0.500000,0.0


In [59]:
# Chequear y limpiar
if(df_buy.isna().values.any()):
    df_buy=df_buy.dropna()
if(df_sell.isna().values.any()):
    df_sell=df_sell.dropna()

In [60]:
# Dividir 
# Preparación de los datos de compra
X_buy = df_buy.drop(columns='target').values
y_buy = df_buy['target'].values
X_buy_train = X_buy
y_buy_train = y_buy
# Preparación de los datos de venta
X_sell = df_sell.drop(columns='target').values
y_sell = df_sell['target'].values
X_sell_train = X_sell
y_sell_train = y_sell

In [61]:
param_grid = {
    'feature_selection__threshold': ['mean', 'median'],
    'xgb__n_estimators': [500],
    'xgb__max_depth': [3],
    'xgb__learning_rate': [0.01],
    'xgb__subsample': [0.6],
    'xgb__colsample_bytree': [0.9],
    'xgb__gamma': [0],
    'scaler__with_mean': [True, False],  # Hiperparámetro del escalador
    'scaler__with_std': [True, False]    # Hiperparámetro del escalador
}


In [62]:
# Crear pipe y entrenar
pipe_buy = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', SelectFromModel(xgb.XGBClassifier(
        use_label_encoder=False, eval_metric='mlogloss'
    ))),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
grid_search_buy = GridSearchCV(
    estimator=pipe_buy,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search_buy.fit(X_buy_train, y_buy_train)
model_buy = grid_search_buy.best_estimator_
pipe_sell = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', SelectFromModel(xgb.XGBClassifier(
        use_label_encoder=False, eval_metric='mlogloss'
    ))),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
grid_search_sell = GridSearchCV(
    estimator=pipe_sell,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search_sell.fit(X_sell_train, y_sell_train)
model_sell = grid_search_sell.best_estimator_

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [63]:
grid_search_buy.best_score_

0.5863110544594694

In [64]:
grid_search_sell.best_score_

0.5766679256368291

In [65]:
update_registered_converter(
    xgb.XGBClassifier,
    "XGBClassifier",
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={'nocl': [False], 'zipmap': [True, False, 'columns']}
)

In [66]:
model_buy_onnx = convert_sklearn(
    model_buy,
    'pipeline_buy_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
model_sell_onnx = convert_sklearn(
    model_sell,
    'pipeline_sell_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
mql5_files_folder = "C:/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files"
with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
    f.write(model_buy_onnx.SerializeToString())
with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
    f.write(model_sell_onnx.SerializeToString())